In [4]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials, space_eval
import matplotlib as plt
import time
import pickle


In [5]:
from sklearn.datasets import make_classification
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, balanced_accuracy_score
from matplotlib.ticker import FormatStrFormatter
import numpy as np
from joblib import dump, load
import json
import sys
import os
from sklearn.model_selection import validation_curve, cross_validate, cross_val_score
import matplotlib.pyplot as plt

In [201]:
def run_nn(algo, 
           path,
           tcol,
           bool_col,
           max_it,
           max_at,
           nodes,
           learning_rate = 0.1,
           tsize=0.2,
           init_temp= 10.0,
           rets=None,
           decay_ob=None,
           psize=None,
           mp=None):
    if decay_ob:
        if decay_ob[0] == "geom":
            schedule = mlrose.GeomDecay(init_temp = init_temp,decay=decay_ob[1])
        elif decay_ob[0] == "exp":
            schedule = mlrose.ExpDecay(init_temp = init_temp, exp_const=decay_ob[1])
    else:
        schedule = None
    
    pdf = pd.read_csv(path)
    X, y = pdf[pdf.columns.difference([tcol])], pdf[tcol]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    sc = StandardScaler()
    sc.fit(X_train[X_train.columns.difference(bool_col)])
    sc.fit(X_test[X_test.columns.difference(bool_col)])
    X_train[X_train.columns.difference(bool_col)] = sc.transform(X_train[X_train.columns.difference(bool_col)])
    X_test[X_test.columns.difference(bool_col)] = sc.transform(X_test[X_test.columns.difference(bool_col)])
    print (X_train.shape)
    print (X_test.shape)
    import six
    import sys
    sys.modules['sklearn.externals.six'] = six
    new = nodes
    if algo == "simulated_annealing":
        nn_model = mlrose.NeuralNetwork(hidden_nodes = list(new),
                                        activation ='relu',
                                        algorithm = algo, 
                                        max_iters = max_it,
                                        bias = True,
                                        schedule = schedule, 
                                        is_classifier = True, 
                                        learning_rate = learning_rate, 
                                        early_stopping = True,                          
                                        max_attempts = max_at, 
                                        random_state = 42,
                                        curve = True)
    elif algo == "random_hill_climb":
        nn_model = mlrose.NeuralNetwork(hidden_nodes = list(new),
                                        activation ='relu',
                                        algorithm = algo, 
                                        max_iters = max_it,
                                        bias = True,
                                        is_classifier = True, 
                                        learning_rate = learning_rate, 
                                        early_stopping = True,                          
                                        max_attempts = max_at, 
                                        random_state = 42,
                                        restarts = rets,
                                        curve = True)
    elif algo == "genetic_alg":
        nn_model = mlrose.NeuralNetwork(hidden_nodes = list(new),
                                        activation ='relu',
                                        algorithm = algo, 
                                        max_iters = max_it,
                                        bias = True,
                                        is_classifier = True, 
                                        learning_rate = learning_rate, 
                                        early_stopping = True,                          
                                        max_attempts = max_at, 
                                        random_state = 42,
                                        pop_size = psize,
                                        mutation_prob = mp,
                                        curve = True)
    else:
        nn_model = mlrose.NeuralNetwork(hidden_nodes = list(new),
                                        activation ='relu',
                                        algorithm = algo, 
                                        max_iters = max_it,
                                        bias = True,
                                        is_classifier = True, 
                                        learning_rate = learning_rate, 
                                        early_stopping = True,                          
                                        max_attempts = max_at, 
                                        random_state = 42,
                                        curve = True)

    cv = ShuffleSplit(n_splits=5, test_size=tsize, random_state=42)
    cv_score = cross_validate(nn_model, X_train, y_train, cv=cv, scoring='f1',return_train_score=True)

    return np.mean(cv_score['test_score']), np.mean(cv_score['fit_time']), nn_model.fitness_curve

In [209]:
def trainer(params):

    if params['alg'] == "simulated_annealing":
        cv_test,cv_fitTime,fc= run_nn(algo = params['alg'],
                                         path = "ionosphere.csv",
                                         tcol = "target",
                                         bool_col = ["feature_0"],
                                         max_it = params['max_iterations'],
                                         max_at = params['max_attempts'],
                                         nodes = params['nodes'],
                                         learning_rate = params['learning_rate'],
                                         init_temp = params['init_temp'],
                                         decay_ob = params['decay_ob'],
                                        )
    elif params['alg'] == "random_hill_climb":
        cv_test,cv_fitTime,fc= run_nn(algo = params['alg'],
                                         path = "ionosphere.csv",
                                         tcol = "target",
                                         bool_col = ["feature_0"],
                                         max_it = params['max_iterations'],
                                         max_at = params['max_attempts'],
                                         nodes = params['nodes'],
                                         learning_rate = params['learning_rate'],
                                         rets = params['restarts']
                                        )
    elif params['alg'] == "genetic_alg":
        cv_test,cv_fitTime,fc= run_nn(algo = params['alg'],
                                         path = "ionosphere.csv",
                                         tcol = "target",
                                         bool_col = ["feature_0"],
                                         max_it = params['max_iterations'],
                                         max_at = params['max_attempts'],
                                         nodes = params['nodes'],
                                         learning_rate = params['learning_rate'],
                                         psize = params['pop_size'],
                                         mp = params['mutation_prob']
                                        )
    elif params['alg'] == "gradient_descent":
        cv_test,cv_fitTime,fc= run_nn(algo = params['alg'],
                                         path = "ionosphere.csv",
                                         tcol = "target",
                                         bool_col = ["feature_0"],
                                         max_it = params['max_iterations'],
                                         max_at = params['max_attempts'],
                                         nodes = params['nodes'],
                                         learning_rate = params['learning_rate']
                                        )
    params['cv_test'] = cv_test
    params['cv_fitTime'] = cv_fitTime

    params['fitness_curve'] = fc
    return {'loss': -cv_test,'status': STATUS_OK, 'params': params}

In [210]:
def sim_nn(max_ev):
    space = {'alg': hp.choice('alg',["simulated_annealing"]),
             'decay_ob': hp.choice('decay_ob',[("geom",0.25),
                                               ("geom",0.5),
                                               ("geom",0.7),
                                               ("geom",0.95),
                                               ("exp",1.58489319e-01),
                                               ("exp",2.51188643e+00),
                                               ("exp",3.98107171e+01)]),
             'init_temp': hp.choice('init_temp',np.arange(1.0,12.0,1.67)),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000]),
             'learning_rate': hp.choice('learning_rate', np.logspace(-4,0,6)),
             'nodes': hp.choice('nodes',[[33],
                                  [16], 
                                  [66],
                                  [33, 33],
                                  [16, 16],
                                  [66, 66],
                                  [33, 33, 33],
                                  [16, 16, 16], 
                                  [66, 66, 66]]
                                )
    }
    trials = Trials()
    best = fmin(trainer, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("nn-annealing.p", "wb"))
    
def rhc_nn(max_ev):
    space = {'alg': hp.choice('alg',["random_hill_climb"]),
             'restarts': hp.choice('restarts',list(range(0,10,1))),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000]),
             'learning_rate': hp.choice('learning_rate', np.logspace(-4,0,6)),
             'nodes': hp.choice('nodes',[[33],
                                  [16], 
                                  [66],
                                  [33, 33],
                                  [16, 16],
                                  [66, 66],
                                  [33, 33, 33],
                                  [16, 16, 16], 
                                  [66, 66, 66]]
                                )
    }
    trials = Trials()
    best = fmin(trainer, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("nn-rhc.p", "wb"))
    
def ga_nn(max_ev):
    space = {'alg': hp.choice('alg',["genetic_alg"]),
             'pop_size': hp.choice('pop_size',[50,100,200,400]),
             'mutation_prob': hp.choice('mutation_prob',[0.001,0.01,0.1,0.2,0.5,0.8,1]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000]),
             'learning_rate': hp.choice('learning_rate', np.logspace(-4,0,6)),
             'nodes': hp.choice('nodes',[[33],
                                  [16], 
                                  [66],
                                  [33, 33],
                                  [16, 16],
                                  [66, 66],
                                  [33, 33, 33],
                                  [16, 16, 16], 
                                  [66, 66, 66]]
                                )
    }
    trials = Trials()
    best = fmin(trainer, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("nn-ga.p", "wb"))
    
def gds_nn(max_ev):
    space = {'alg': hp.choice('alg',["gradient_descent"]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000]),
             'learning_rate': hp.choice('learning_rate', np.logspace(-4,0,6)),
             'nodes': hp.choice('nodes',[[33],
                                  [16], 
                                  [66],
                                  [33, 33],
                                  [16, 16],
                                  [66, 66],
                                  [33, 33, 33],
                                  [16, 16, 16], 
                                  [66, 66, 66]]
                                )
    }
    trials = Trials()
    best = fmin(trainer, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("nn-gds.p", "wb"))

In [212]:
EVAL = 100
gds_nn(EVAL)
ga_nn(EVAL)
rhc_nn(EVAL)
sim_nn(EVAL)

(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                               

(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                               

(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                

(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                

(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                               

...

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/

 37%|████████████████████████████▊                                                 | 37/100 [03:35<05:55,  5.64s/trial, best loss: -0.8750576441102756]

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/tekram/venv/lib/python3.7/site-packages/mlrose/activation.py:77: RuntimeWarning: overflow encountered in exp
  fx = 1/(1 + np.exp(-x))

/home/

(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                               

(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                                                                                              
(71, 33)                                                                                                                                               
(280, 33)                                                                               